In [ ]:
from sys import byteorder

byteorder = 'big'


以下サイトを参考にしました。  
https://note.com/cider00/n/nf2b49924b13e

In [ ]:
def extract_between_strings(filename, string_a, string_b):
    """
    extract strings between string_a and string_b from file
    """
    results = []
    capture = False
    buffer = []
    
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if string_a in line:
                capture = True
                buffer.append(line)
                continue
            
            if capture:
                buffer.append(line)
                if string_b in line:
                    capture = False
                    results.append(''.join(buffer))
                    buffer = []

    return results

In [ ]:
def to_hex(vcs_string, suffix_to_remove):
    # assume string as something like =89=C6=8C=76=95=EB\nLOCATION
    a = vcs_string.replace("=","")
    b = a.replace("\n","")
    c = b.replace(suffix_to_remove,"")
    return c

def decode_string(preprocessed_string, encoding="shift-jis"):
    a = int(preprocessed_string, 16)
    b = a.to_bytes(len(str(a)), 'big')
    d = ""
    try:
        c = b.decode(encoding=encoding)
        d = c.strip("\x00")
    except UnicodeDecodeError as e:
        try:
            c = b.decode(encoding="cp932")
            d = c.strip("\x00")
        except UnicodeDecodeError as e2:
            print("catch UnicodeDecodeError. can't decode with shift-jis and cp932")
    finally:
        return d
    
        

#### DESCRITIONとCATEGORIESに囲まれた部分をまず取り出す

In [ ]:
filename = 'data/[your_file].vcs'
string_a = 'DESCRIPTION'
string_b = 'CATEGORIES'
extracted_strings_body = extract_between_strings(filename, string_a, string_b)

# 出力（必要に応じて）
for s in extracted_strings_body:
    print(s)

In [ ]:
# 題名部分
string_c = 'SUMMARY'
string_d = 'LOCATION'
extracted_strings_title = extract_between_strings(filename, string_c, string_d)



### 1行ごとに抜き出していく

In [ ]:
text_dic = {}

OUTPUT_FOLDER = "output"

import os
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for title, body in zip(extracted_strings_title, extracted_strings_body):
    # 中身は:で区切った二つ目
    title_str = title.split(":")[1]
    body_str = body.split(":")[1]
    # titleはLOCATION, bodyはCAGORIRIESを消す
    title_hex = to_hex(title_str, "LOCATION")
    if title_hex == "" : continue
    body_hex = to_hex(body_str, "CATEGORIES")
    # 中身がなくてもタイトルだけのtextファイルを保存する
    if body_hex == "" :
        title_text = decode_string(title_hex)
        text_dic[title_text] = ""
    else:
        body_text = decode_string(body_hex)
        title_text = decode_string(title_hex)
        text_dic[title_text] = body_text

        


print(text_dic)

    

In [ ]:
for title in text_dic:
    with open(OUTPUT_FOLDER + "/" + title + ".txt", mode="w") as f:
        f.write(text_dic[title])